In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from sklearn.linear_model import LinearRegression, RANSACRegressor

from glob import glob
import seaborn as sns


from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist, cosine
from scipy.ndimage import convolve1d
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import normalize
from joblib import Parallel, delayed

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

from netCDF4 import Dataset, num2date, date2num
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

In [2]:
ds = xr.open_dataset('classified_data.nc')

In [3]:
data = ds.to_dataframe()

In [4]:
df = data.copy()

In [5]:
list(df.keys())

['Wind Direction (TAO)',
 'WDIR_QC',
 'WDIR_DM',
 'Wind Speed (TAO)',
 'WSPD_QC',
 'WSPD_DM',
 'Zonal Wind Speed (TAO)',
 'Meridional Wind Speed (TAO)',
 'Sea Surface Temperature (TAO)',
 'SST_QC',
 'SST_DM',
 'Relative Humidity (TAO)',
 'RELH_QC',
 'RELH_DM',
 'Air Temperature (TAO)',
 'AIRT_QC',
 'AIRT_DM',
 'Neutral Wind Speed at 10m (TAO)',
 'Wind Speed at 10m (TAO)',
 'mean_WSPD',
 'std_WSPD',
 'mean_WSPD_10N',
 'std_WSPD_10N',
 'mean_SST',
 'std_SST',
 'mean_AIRT',
 'std_AIRT',
 'mean_RELH',
 'std_RELH',
 'mean_U10N_x',
 'std_U10N_x',
 'mean_U10N_y',
 'std_U10N_y',
 'mean_cosWDIR',
 'std_cosWDIR',
 'mean_sinWDIR',
 'std_sinWDIR',
 'LATITUDE',
 'LONGITUDE',
 'TAO_TIME',
 'Deployment index',
 'Wind Speed (QuikSCAT)',
 'Wind Direction (QuikSCAT)',
 'rain_impact',
 'flags',
 'eflags',
 'nudge_wind_speed',
 'nudge_wind_direction',
 'retrieved_wind_speed_uncorrected',
 'cross_track_wind_speed_bias',
 'atmospheric_speed_bias',
 'wind_obj',
 'ambiguity_speed',
 'ambiguity_direction',
 'a

In [6]:
np.min(df['LONGITUDE']), np.max(df['LONGITUDE'])

(165.0, 265.0)

In [7]:
def plotTimeSeriesByClusterLabel(df, lat, lon, ax1, ax2, cluster_label=0, color='blue'):
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == (lon+360)%360]
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'

    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'

    

    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        ax1.axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        ax2.axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    #ax1.scatter(subDF['time'], y1, s=2, label = cluster_label)#, c = subDF['label'])
    #ax2.scatter(subDF['time'], y2, s=2, label = cluster_label)#, c = subDF['label'])

    ax1.scatter(dates, y1, s=3, label = cluster_label, c = color)
    ax2.scatter(dates, y2, s=3, label = cluster_label, c = color)
    ax1.set_title(f'{lat:02d}{latUnits} {lon:03d}{lonUnits} speed Diff')
    ax2.set_title(f'{lat:02d}{latUnits} {lon:03d}{lonUnits} direction Diff')

def plotTimeSeriesAllClusterLabel(df, lat, lon, ax1, ax2):
    subDF = df.loc[df['lat'] == lat]
    subDF = subDF.loc[subDF['lon'] == lon%360]
    
    
    dates = num2date(subDF['time'], 'seconds since 1990-01-01 00:00:00')

    y1 = subDF['speedDiff']
    y2 = subDF['directionDiff']
    
    ax1.scatter(subDF['time'], y1, s=2, c = subDF['Deployment Classifier'], cmap='prism')
    ax2.scatter(subDF['time'], y2, s=2, c = subDF['Deployment Classifier'], cmap='prism')
    

In [8]:
def plotSusanLike(SUBDF, lat, lon, suffix='all'):

    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['mean_U10N_x'])
    Uy_TAO = np.array(subDF['mean_U10N_y'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
    
    
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20,8))
    
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['ratio'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[0].plot(ssubDF['QS_TIME'][~mask], Y)
    
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['angle_diff'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[1].plot(ssubDF['QS_TIME'][~mask], Y)
        
    
    
    
    ax = axes[0]
    ax.scatter(subDF['QS_TIME'], ratio, c = 'r', s = 0.02)
    
    ax = axes[1]
    ax.scatter(subDF['QS_TIME'], angle, c = 'r', s = 0.02)
    
    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        axes[0].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[1].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    axes[0].set_ylim(0,2)
    axes[1].set_ylim(-50,50)
    
    axes[0].set_title('Ratio of wind speeds (QS/TAO)')
    axes[1].set_title('Difference in wind direction (QS - TAO)')
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    title = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}.png'
    plt.savefig(title, dpi = 100)
    plt.close()



In [ ]:
def plotSusanLikeWithCorrectedAngle(SUBDF, lat, lon, suffix='all'):

    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['mean_U10N_x'])
    Uy_TAO = np.array(subDF['mean_U10N_y'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
        
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20,8))
    
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['ratio'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[0].plot(ssubDF['QS_TIME'][~mask], Y)
    
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['angle_diff'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[1].plot(ssubDF['QS_TIME'][~mask], Y)
        
    
    
    
    ax = axes[0]
    ax.scatter(subDF['QS_TIME'], ratio, c = 'r', s = 0.02)
    
    ax = axes[1]
    ax.scatter(subDF['QS_TIME'], angle, c = 'r', s = 0.02)
    
    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        axes[0].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[1].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    axes[0].set_ylim(0,2)
    axes[1].set_ylim(-50,50)
    
    axes[0].set_title('Ratio of wind speeds (QS/TAO)')
    axes[1].set_title('Difference in wind direction (QS - TAO)')
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    title = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}.png'
    plt.savefig(title, dpi = 100)
    plt.close()



In [9]:
def correctAngleLinReg(SUBDF, lat, lon, suffix='all'):

    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['mean_U10N_x'])
    Uy_TAO = np.array(subDF['mean_U10N_y'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
    
    
    #fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20,8))
    list_all_dep_df = [] #pd.DataFrame()
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['ratio'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue

        ssubDF = ssubDF.loc[~mask]
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        

        mask1 = mask
        
        #axes[0].plot(ssubDF['QS_TIME'][~mask], Y)
    
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['angle_diff'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue

        ssubDF = ssubDF.loc[~mask]
        
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)

        ssubDF['corrected_angle_diff'] = y-Y
        
        #axes[1].plot(ssubDF['QS_TIME'][~mask], Y)
        list_all_dep_df.append(ssubDF)

    allDF = pd.concat(list_all_dep_df)
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    fname = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}_data.csv'
    allDF.to_csv(fname, index=False)


In [11]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)

In [114]:
for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLike(df, lat, lon, suffix='all')

/tmp/ipykernel_2049744/4235400786.py:18: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_2049744/4235400786.py:18: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_2049744/4235400786.py:18: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO


In [116]:
subDF = df.loc[df['label'] == 0]
for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLike(subDF, lat, lon, suffix='good')

/tmp/ipykernel_2049744/4235400786.py:18: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO


In [12]:
subDF = df.loc[df['predicted_rain_flag'] == 0]
for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    correctAngleLinReg(subDF, lat, lon, suffix='dry')

/tmp/ipykernel_282787/553249461.py:18: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_282787/553249461.py:18: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_282787/553249461.py:18: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO


In [118]:
subDF = df.loc[df['predicted_rain_flag'] == 0]
subDF = subDF.loc[subDF['label'] == 0]
for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLike(subDF, lat, lon, suffix='dryAndGood')

/tmp/ipykernel_2049744/4235400786.py:18: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
